Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Installation and configuration
This notebook configures the notebooks in this tutorial to connect to an Azure Machine Learning (AML) Workspace.  You can use an existing workspace or create a new one.

## Prerequisites

If you have already completed the prerequisites and selected the correct Kernel for this notebook, the AML Python SDK is already installed. Let's check the AML SDK version.

In [ ]:
import azureml.core
from ado_ai_utils.aml_configuration.register_datastores import register_blob_datastore, register_sql_datastore
from ado_ai_utils.aml_configuration.register_samples import register_ai_feedback_sample, register_feedback_sample, \
    register_improvements_sample
from ado_ai_utils.aml_configuration.utils import load_configuration, get_or_create_workspace

print("AML SDK Version:", azureml.core.VERSION)

## Set up your Azure Machine Learning workspace

To create or access an Azure ML Workspace, you will need the following information:

* Your subscription id
* A resource group name
* A name for your workspace
* A region for your workspace

**Note**: As with other Azure services, there are limits on certain resources like cluster size associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

If you have a workspace created already, you need to get your subscription and workspace information. You can find the values for those by visiting your workspace in the [Azure portal](http://portal.azure.com). If you don't have a workspace, the create workspace command in the next section will create a resource group and a workspace using the names you provide.

Replace the values in the following cell with your information. If you would like to use service principal authentication as described [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb) make sure you provide the optional values as well. 

In [ ]:
cfg = load_configuration("workspace_conf.yml")

In [ ]:
# Azure resources
subscription_id = cfg['subscription_id']
resource_group = cfg['resource_group']  
workspace_name = cfg['workspace_name']  
workspace_region = cfg['workspace_region']
image_name = cfg['image_name']  # e.g. image_name = "{{cookiecutter.image_name}} (avoid underscore in names)"

sql_server_name=cfg['sql_server_name']    # Name of Azure SQL server
sql_database_name=cfg['sql_database_name']  # Name of Azure SQL database
sql_username=cfg['sql_username']        # The username of the database user to access the database.
sql_password=cfg['sql_password']        # The password of the database user to access the database.

datastore_rg = cfg['datastore_rg']
container_name=cfg['container_name']    # Name of Azure blob container
account_name=cfg['account_name']    # Storage account name
account_key=cfg['account_key']         # Storage account key

tenant_id = "YOUR_TENANT_ID" # Optional for service principal authentication
username = "YOUR_SERVICE_PRINCIPAL_APPLICATION_ID" # Optional for service principal authentication
password = "YOUR_SERVICE_PRINCIPAL_PASSWORD" # Optional for service principal authentication
storageConnString = "YOUR_STORAGE_CONNECTION_STRING"


In [ ]:
sql_datastore_name = "ado_sql_datastore"
blob_datastore_name="ado_blob_datastore"

#### Create the workspace
This cell will create an AML workspace for you in a subscription, provided you have the correct permissions.

This will fail when:
1. You do not have permission to create a workspace in the resource group
2. You do not have permission to create a resource group if it's non-existing.
2. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription

If workspace creation fails, please work with your IT admin to provide you with the appropriate permissions or to provision the required resources. If this cell succeeds, you're done configuring AML!  


In [ ]:
ws = get_or_create_workspace(workspace_name, subscription_id, resource_group, workspace_region)

Let's check the details of the workspace.

In [ ]:
ws.get_details()

In [ ]:
register_sql_datastore(ws, sql_datastore_name, sql_server_name, sql_database_name, sql_username, sql_password)

In [ ]:
blob_datastore = register_blob_datastore(ws, blob_datastore_name, container_name, account_name, account_key, datastore_rg)

In [ ]:
register_ai_feedback_sample(blob_datastore, ws)

In [ ]:
register_feedback_sample(blob_datastore, ws)

In [ ]:
register_improvements_sample(blob_datastore, ws)


You are now ready to move on to the [AutoML Local](01_AutoML_Local.ipynb) notebook.